In [1]:
import math
import torch
import gpytorch

import pyro
pyro.enable_validation(True)

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
%matplotlib inline

In [2]:
%pdb

Automatic pdb calling has been turned ON


## Introduction

In this notebook, we how to train a functional kernel learning model, training on a simple function. We'll be modeling the function

\begin{align*}
  y &= \sin(2\pi x) + \epsilon \\
  \epsilon &\sim \mathcal{N}(0, 0.2)
\end{align*}

with 11 training examples, and testing on 51 test examples.

For more mathematical detail, please see the paper Functional Distributions over Kernels (NeurIPS 2019).

**Note:** this notebook is not necessarily intended to teach the mathematical background of Gaussian processes, but rather how to train a simple one and make predictions in GPyTorch. For a mathematical treatment, Chapter 2 of Gaussian Processes for Machine Learning provides a very thorough introduction to GP regression (this entire text is highly recommended): http://www.gaussianprocess.org/gpml/chapters/RW2.pdf


#### Set up the GPyTorch Model with Spectral GP kernel
Using the same framework as standard GPyTorch models, we merely drop-in the spectral GP kernel as the covar module

The `initialize_from_data` method does some pre-training on the latent model using the log-periodogram of data as training targets.

For specifics on the components of GPyTorch models we refer to the [GPyTorch Documentation](https://gpytorch.readthedocs.io/en/latest/index.html)

In [3]:
class SpectralModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, **kwargs):
        super(SpectralModel, self).__init__(train_x, train_y, likelihood)

        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.SpectralGPKernel(**kwargs)

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


#### Generate training data and Build GP Model
Generate points in [0, 5] and a sine wave to serve as the response, then split into training and test data.

Pass this data into the GP model from above along with a likelihood

In [4]:
nx = 200
split = 150
full_x = torch.linspace(0, 5, nx).cuda()
full_y = torch.sin(2 * full_x)

train_x = full_x[:split]
train_y = full_y[:split]

test_x = full_x[(split - nx):]
test_y = full_y[(split - nx):]

### Define latent model

In [5]:
class FKL_GPModel(gpytorch.likelihoods.GaussianLikelihood):
    def __init__(self, train_x, train_y, data_likelihood, **kwargs):
        super(FKL_GPModel, self).__init__()
        
        self._max_plate_nesting = 1
        self.data_likelihood = data_likelihood
        self.data_model = SpectralModel(train_x, train_y, self.data_likelihood, 
                                        **kwargs)
        self.train_x = train_x
        self.train_y = train_y
           
    @staticmethod
    def gplink_function(f):
        ### gp link function is inside the kernel
        return f.view(-1)
    
    def forward(self, function_samples, *params, **kwargs):
        latent_gp = self.gplink_function(function_samples)

        self.data_model.covar_module.set_latent_params(latent_gp)
        
        self.data_model.train()
        
        if not self.training:
            self.data_model.eval()
            dist = self.data_model(*params, **kwargs)
        else:
            dist = self.data_likelihood(self.data_model(self.train_x))
        
            with torch.no_grad():
                print('forwards into dist: ', dist.log_prob(self.train_y))
        return dist

In [6]:
#omega = torch.arange(1e-4, 1., 0.01)

data_lh = gpytorch.likelihoods.GaussianLikelihood(noise_prior=gpytorch.priors.SmoothedBoxPrior(1e-8, 1e-4)).cuda()
data_gp = FKL_GPModel(train_x, train_y, data_lh, train_inputs = train_x, register_latent_params=False).cuda()
data_gp.train()
data_lh.train()


GaussianLikelihood(
  (quadrature): GaussHermiteQuadrature1D()
  (noise_covar): HomoskedasticNoise(
    (noise_prior): SmoothedBoxPrior()
    (raw_noise_constraint): GreaterThan(1.000E-04)
  )
)

In [11]:
omega = data_gp.data_model.covar_module.omega.data.cuda()


In [12]:
from gpytorch.means import QuadraticMean
from gpytorch.priors import NormalPrior
from gpytorch.constraints import LessThan
from gpytorch.kernels import ScaleKernel, MaternKernel, GridKernel
from gpytorch.models import PyroVariationalGP
from gpytorch.distributions import MultivariateNormal
from gpytorch.variational import CholeskyVariationalDistribution, VariationalStrategy

# we construct a Scale(Matern 3/2) kernel with a quadratic mean by default
class PyroGPModel(PyroVariationalGP):
    def __init__(self, train_x, train_y, likelihood, mean_module, covar_module):
        # Define all the variational stuff
        variational_distribution = CholeskyVariationalDistribution(
            num_inducing_points=int(train_x.numel())
        )
        variational_strategy = VariationalStrategy(
            self, train_x, variational_distribution
        )

        super(PyroGPModel, self).__init__(variational_strategy, likelihood, num_data=train_x.numel())
        self.mean_module = mean_module
        self.covar_module = covar_module

    def forward(self, x):
        mean = self.mean_module(x)
        covar = self.covar_module(x)
        return MultivariateNormal(mean, covar)

# construct by default a LogRBF prior on the latent spectral GP
latent_mean = QuadraticMean()#.to(device)
latent_mean.register_prior(
    "bias_prior",
    prior=NormalPrior(torch.zeros(1), 100.0 * torch.ones(1), transform=None),
    param_or_closure="bias"
)
latent_mean.register_constraint("quadratic_weights", constraint=LessThan(upper_bound=0.0))
latent_mean.register_prior(
    "quadratic_weights_prior",
    prior=NormalPrior(
        torch.zeros(1),
        100.0 * torch.ones(1),
        transform=torch.nn.functional.softplus,
    ),
    param_or_closure="quadratic_weights"
)

latent_covar = ScaleKernel(
    MaternKernel(nu=1.5, lengthscale_prior=NormalPrior(torch.zeros(1), torch.ones(1), transform=torch.exp)),
    outputscale_prior=NormalPrior(torch.zeros(1), torch.ones(1), transform=torch.exp),
)
# latent_covar = GridKernel(latent_covar, grid=self.omega.unsqueeze(-1))

latent_model = PyroGPModel(
    omega, None, data_gp, mean_module=latent_mean, covar_module=latent_covar
).cuda()

In [13]:
data_gp.train();
latent_model.train();

In [ ]:
cols = ['red', 'blue', 'green', 'purple', 'orange']
for sample_num in range(5):
    latent_sample = latent_model(omega).rsample()
    
    prior_dist = data_gp(latent_sample, train_x)
    prior_preds = prior_dist.sample(torch.Size((3,)))
    
    plt.plot(train_x, prior_preds.data.t().numpy(), color = cols[sample_num], alpha = 0.3)

RuntimeError: expected device cpu and dtype Float but got device cuda:0 and dtype Float

> /home/wesley/Documents/Code/wjm_gpytorch/gpytorch/kernels/spectral_gp_kernel.py(188)compute_kernel_values()
    186 
    187         # numerically compute integral in expanded space
--> 188         integrand = density * torch.cos(2.0 * math.pi * self.omega * expanded_tau)
    189 
    190         # compute trapezoidal rule for now



In [ ]:
# train the model
from pyro import optim

base_lr = 1e-2
iter_print = 5

print('Basic lr for most of parameters: {}'.format(base_lr))

# set learning rates for different hyperparameters
def per_param_callable(module_name, param_name):
    if param_name == 'covar_module.base_kernel.raw_lengthscale':
        return {"lr": 0.1}
    elif param_name == 'variational_strategy.variational_distribution.variational_mean':
        return {"lr": base_lr}
    else:
        return {"lr": base_lr}

# Use the adam optimizer
optimizer = optim.Adam(per_param_callable)

pyro.clear_param_store() # clean run

losses, rbf, means = [], [], []

means.append(latent_model.variational_strategy.variational_distribution.\
             variational_mean.detach().cpu().numpy()) #save initial mean

def train(num_iter=1500):
    elbo = pyro.infer.Trace_ELBO(num_particles=1, vectorize_particles=True)
    svi = pyro.infer.SVI(latent_model.model, latent_model.guide, optimizer, elbo)
    latent_model.train()

    for i in range(num_iter):
        latent_model.zero_grad()
        loss = svi.step(omega, train_y)
        print(latent_model(omega).log_prob(data_gp.data_model.covar_module.latent_params).sum(), 
             data_gp.data_model.covar_module.latent_params.norm())
        losses.append(loss)
        rbf.append(latent_model.covar_module.base_kernel.lengthscale.item())
        if not (i + 1) % iter_print:
            print('Iter {}/{} - Loss: {:.3}   lengthscale: {:.3}'.format(
                i + 1, num_iter, loss,
                latent_model.covar_module.base_kernel.lengthscale.item(),
            ))
            means.append(latent_model.variational_strategy.variational_distribution.\
                         variational_mean.detach().cpu().numpy())
        
%time train()

In [ ]:
# prot loss function and kernel length
fig, (loss, kern) = plt.subplots(1,2,figsize=(12,2))
loss.plot(losses)
loss.set_xlabel("Epoch")
loss.set_ylabel("Loss")
kern.plot(rbf)
kern.set_xlabel("Epoch")
kern.set_ylabel("Kernel scale parameter")
fig.show()

In [ ]:
test_x = torch.linspace(0, 15, 500)

In [ ]:
data_gp.eval()

latent_model.eval()

with torch.no_grad():
    output_dist = latent_model(omega)
    
samples = output_dist.sample(torch.Size((20,)))

In [ ]:
list(latent_model.named_parameters())

In [ ]:
for i, sample in enumerate(samples):
    if i is 0:
        plt.plot(omega, sample.exp(), color = 'black', alpha = 0.3, label = 'S(w)')
    else:
        plt.plot(omega, sample.exp(), color = 'black', alpha = 0.3)

plt.xlabel('omega')
plt.ylabel('S(omega)')
plt.grid()
plt.legend()

In [ ]:
for i, sample in enumerate(samples):
    with gpytorch.settings.fast_pred_var():
        pred_dist = data_gp(sample.data, test_x)
    
    pred_mean = pred_dist.mean.data.numpy()
    lower, upper = pred_dist.confidence_region()
    
    if i is 0:
        plt.plot(test_x, pred_mean, color = 'blue', label = 'Predictive Mean')
    else:
        plt.plot(test_x, pred_mean, color = 'blue')
    
    plt.fill_between(test_x.numpy(), lower.data.numpy(), upper.data.numpy(), alpha = 0.03, color = 'blue')
    
plt.plot(train_x, train_y, marker = 'x', color = 'black', label = 'Training Data')
plt.grid()
plt.xlabel('x')
plt.ylabel('y')
plt.legend()

In [ ]:
covar_output = data_gp.data_model.covar_module(torch.zeros(1), torch.linspace(0., 20., 100)).evaluate()

In [ ]:
plt.plot(torch.linspace(0., 20., 100), covar_output.view(-1).numpy())

In [ ]:
covar_output